In [ ]:
# Install OpenNMT-py 3.x
!pip3 install OpenNMT-py
!git clone https://github.com/OpenNMT/OpenNMT-py.git
!cd OpenNMT-py
%cd /content/OpenNMT-py
!python setup.py install
!pip install -r requirements.opt.txt
!pip install tensorrt

In [ ]:
%cd /content/drive/MyDrive/Dissertation/NMT/nmt

/content/drive/MyDrive/Dissertation/NMT/nmt


In [ ]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: TildeMODEL.en-mt.en-filtered.en.subword.train
        path_tgt: TildeMODEL.en-mt.mt-filtered.mt.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: TildeMODEL.en-mt.en-filtered.en.subword.dev
        path_tgt: TildeMODEL.en-mt.mt-filtered.mt.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.fren

# Stop training if it does not imporve after n validations
early_stopping: 4

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 5000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 25000

# Default: 10000 - Run validation after n steps
valid_steps: 5000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 4000
report_every: 2500

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
# Find the number of CPUs/cores on the machine
!nproc --all

2


In [ ]:
!pip install tensorrt

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17281 sha256=26bcfff444a17083d2e4cca3ae73f7ebd9e2e103d4565811ce291d1e17ee2e99
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built tensorrt


In [ ]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

2023-10-24 21:09:57.631821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-24 21:09:57.631887: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-24 21:09:57.631933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-24 21:09:57.644862: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 21:09:58.930255: W tensorflow/compiler/

# Switch Runtime to GPU

In [1]:
# Check if the GPU is active
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-464ea213-b1fa-6321-cd58-b55edd191298)


In [3]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git
!cd OpenNMT-py
%cd /content/OpenNMT-py
!python setup.py install
!pip install -r requirements.opt.txt
!pip install tensorrt

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 21212, done.
remote: Counting objects: 100% (2060/2060), done.
remote: Compressing objects: 100% (982/982), done.
remote: Total 21212 (delta 1187), reused 1814 (delta 1040), pack-reused 19152
Receiving objects: 100% (21212/21212), 309.21 MiB | 30.38 MiB/s, done.
Resolving deltas: 100% (15120/15120), done.
/content/OpenNMT-py
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/l

In [4]:

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla T4
Free GPU memory: 14998.8125 out of: 15101.8125


In [ ]:
!pwd

/content/OpenNMT-py


In [5]:
%cd /content/drive/MyDrive/Dissertation/NMT/nmt

/content/drive/MyDrive/Dissertation/NMT/nmt


In [6]:
# Train the NMT model
# !onmt_train -config config.yaml
!onmt_train -config config.yaml -train_from models/model.fren_step_15000.pt

2023-10-25 21:01:23.318259: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-25 21:01:23.318324: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-25 21:01:23.318360: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-25 21:01:23.326402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-25 21:01:24.782890: W tensorflow/c

In [ ]:
!onmt_translate -model models/model.fren_step_10000.pt -src TildeMODEL.en-mt.en-filtered.en.subword.test -output TildeMODEL.mt.translated -gpu 0 -min_length 1

2023-10-24 20:49:44.960252: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-24 20:49:44.960318: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-24 20:49:44.960360: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-24 20:49:44.971665: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 20:49:47.323894: W tensorflow/c

In [ ]:
!head -n 5 TildeMODEL.mt.translated

▁Huwa ▁rrakkomandat ▁ukoll ▁li ▁l - miri ▁korrispondenti ▁jiġu ▁inklużi ▁fil - programmi ▁nazzjonali ▁ta ’ ▁riforma .
▁L - Istati ▁Membri ▁għandhom ▁jimpenjaw ▁ruħhom ▁li ▁jiżguraw , ▁sa ▁mhux ▁aktar ▁tard ▁mit - tmiem ▁ta ’ ▁l - 2 0 0 7 , ▁li ▁jiġu ▁stabbiliti ▁skemi ▁adegwati ▁biex ▁itejbu ▁l - infrastruttura ▁fil - Punti ▁ta ’ ▁Rispons ▁Pubbliku ▁( PSAP ) ▁għall - ipproċessar ▁tal - post ▁u ▁informazzjoni ▁oħra ▁ta ’ ▁l - eCall s ▁li ▁joriġinaw ▁mill - vetturi .
▁Huwa ▁importanti ▁li ▁l - Grupp ▁ta ’ ▁Esperti ▁ta ’ ▁Livell ▁Għoli ▁jipprovdi ▁opportunitajiet ▁għal ▁djalogu ▁u ▁konsultazzjoni ▁mal - imsieħba ▁soċjali ▁u ▁l - organizzazzjoni jiet ▁tas - soċjetà ▁ċivili .
▁L - ebda ▁element ▁ta ’ ▁distribuzzjoni ▁ġeografika ▁ma ▁jkun ▁previst .
▁Mill - 1 ▁ta ’ ▁Jannar ▁ 2 0 0 9 , ▁l - Istati ▁Membri ▁għandhom ▁jesiġu ▁li ▁l - fornituri ▁ta ’ ▁fjuwils ▁għat - trasport ▁bit - triq ▁u ▁makkinarju ▁mobbli ▁mhux ▁tat - triq ▁li ▁jitqiegħdu ▁fis - suq , ▁jimmonitorjaw ▁u ▁jirrappurtaw ▁l - em

In [ ]:
!pip3 install --upgrade -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00


In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py target.model TildeMODEL.mt.translated

Done desubwording! Output: TildeMODEL.mt.translated.desubword


In [ ]:
!head -n 5 TildeMODEL.mt.translated.desubword

Huwa rrakkomandat ukoll li l-miri korrispondenti jiġu inklużi fil-programmi nazzjonali ta’ riforma.
L-Istati Membri għandhom jimpenjaw ruħhom li jiżguraw, sa mhux aktar tard mit-tmiem ta’ l-2007, li jiġu stabbiliti skemi adegwati biex itejbu l-infrastruttura fil-Punti ta’ Rispons Pubbliku (PSAP) għall-ipproċessar tal-post u informazzjoni oħra ta’ l-eCalls li joriġinaw mill-vetturi.
Huwa importanti li l-Grupp ta’ Esperti ta’ Livell Għoli jipprovdi opportunitajiet għal djalogu u konsultazzjoni mal-imsieħba soċjali u l-organizzazzjonijiet tas-soċjetà ċivili.
L-ebda element ta’ distribuzzjoni ġeografika ma jkun previst.
Mill-1 ta’ Jannar 2009, l-Istati Membri għandhom jesiġu li l-fornituri ta’ fjuwils għat-trasport bit-triq u makkinarju mobbli mhux tat-triq li jitqiegħdu fis-suq, jimmonitorjaw u jirrappurtaw l-emissjonijiet tal-gassijiet b’effett ta’ serra tul iċ-ċiklu tal-ħajja minn dawk il-fjuwils.
